In [1]:
import sys
import os
import pathlib

parent_dir = pathlib.Path.cwd().parent
sys.path.append(str(parent_dir))

#from multiprocessing import Pool
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score

from tqdm.notebook import tqdm
from deepdiff import DeepDiff


from utils.partition import weight_update_statistics
from utils.flower_detection import mal_agents_update_statistics
from utils.data_crunch import load_meta_metrics, metrics_to_WL


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
Rs = [0, 3, 10, 30, 100, 300, 1_000, 3_000]
SERVER_ROUNDS = 20
kappas = np.round(np.concatenate((np.arange(0, 1.1, 0.1), np.arange(5,55,5))),3)
print(f"{kappas=}, {len(kappas)=}")
# R= 0
#meta_metrics = {}
#meta_metrics = load_meta_metrics(R)
    
#for i in range(SERVER_ROUNDS):
#    # PREVIOUS RESULTS!!!
#    with open(f'../outputs/reproducibility_detection2/{i+1}_metrics.pkl', 'rb') as handle: 
#        metrics = pickle.load(handle)
#    meta_metrics[i] = metrics
    

# Start loop
l_classifier_res = []

for iR, R in enumerate(Rs):
    print(f"{iR}/{len(Rs)}")
    meta_metrics = {}
    meta_metrics = load_meta_metrics(R)
    
    results = {}
    WL_all = [metrics_to_WL(meta_metrics[rnd]) for rnd in meta_metrics.keys()]

    for kappa in tqdm(kappas):
        result_at_kappa = []
        for rnd in meta_metrics.keys():
            WL = WL_all[rnd]
            detected_round = weight_update_statistics(WL, kappa=kappa)
            result_at_kappa.append(detected_round)
        results[str(np.round(kappa, 1))] = result_at_kappa
    results_uni = {}
    for kappa in results.keys():
        results_uni[kappa] = np.concatenate(results[kappa])
        
    if R == 0:
        Y_true = [0] * 200
    else:    
        Y_true = [0, 0, 0, 1, 0, 0, 0, 0 ,0 ,0] * 20
    
    for kappa in results_uni.keys():
        accu = accuracy_score(Y_true, results_uni[kappa])
        cmtx = confusion_matrix(Y_true, results_uni[kappa])
        if all(results_uni[kappa] == 0) and all(np.array(Y_true) == 0):
            TN = cmtx[0,0]
            FN = TP = FP = 0 
        else:
            TN = cmtx[0,0]
            FN = cmtx[1,0]
            TP = cmtx[1,1]
            FP = cmtx[0,1]
        l_classifier_res.append([R, float(kappa), accu, TN, FN, TP, FP])
    


kappas=array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        5. , 10. , 15. , 20. , 25. , 30. , 35. , 40. , 45. , 50. ]), len(kappas)=21
0/8


  0%|          | 0/21 [00:00<?, ?it/s]

1/8


  0%|          | 0/21 [00:00<?, ?it/s]

2/8


...

  0%|          | 0/21 [00:00<?, ?it/s]

6/8


  0%|          | 0/21 [00:00<?, ?it/s]

7/8


  0%|          | 0/21 [00:00<?, ?it/s]

In [3]:
df_classifier = pd.DataFrame(l_classifier_res, columns = ["R", "kappa", "accu", "TN", "FN", "TP", "FP"])
df_classifier.head(200)

,R,kappa,accu,TN,FN,TP,FP
0,0,0.0,0.000,0,0,0,200
1,0,0.1,0.015,3,0,0,197
2,0,0.2,0.040,8,0,0,192
3,0,0.3,0.085,17,0,0,183
4,0,0.4,0.195,39,0,0,161
5,0,0.5,0.370,74,0,0,126
6,0,0.6,0.560,112,0,0,88
7,0,0.7,0.690,138,0,0,62
8,0,0.8,0.835,167,0,0,33
9,0,0.9,0.910,182,0,0,18


In [4]:
df_classifier.to_csv(f'/home/salesort/Documents/644_CMPUT/res_analysis/df_classifier_R.csv', index = False)

---

## Tests

In [31]:
# 1min 40s
results = {}
WL_all = [metrics_to_WL(meta_metrics[rnd]) for rnd in meta_metrics.keys()]

for kappa in tqdm(kappas):
    result_at_kappa = []
    for rnd in meta_metrics.keys():
        WL = WL_all[rnd]
        detected_round = weight_update_statistics(WL, kappa=kappa)
        result_at_kappa.append(detected_round)
    results[str(np.round(kappa, 1))] = result_at_kappa
    

  0%|          | 0/21 [00:00<?, ?it/s]

In [63]:
results_uni = {}
for kappa in results.keys():
    results_uni[kappa] = np.concatenate(results[kappa])
    
if R == 0:
    Y_true = [0] * 200
else:    
    Y_true = [0, 0, 0, 1, 0, 0, 0, 0 ,0 ,0] * 20


l_classifier_res = []

for kappa in results_uni.keys():
    accu = accuracy_score(Y_true, results_uni[kappa])
    cmtx = confusion_matrix(Y_true, results_uni[kappa])
    if all(results_uni[kappa] == 0) and all(np.array(Y_true) == 0):
        TN = cmtx[0,0]
        FN = TP = FP = 0 
    else:
        TN = cmtx[0,0]
        FN = cmtx[1,0]
        TP = cmtx[1,1]
        FP = cmtx[0,1]
    l_classifier_res.append([R, float(kappa), accu, TN, FN, TP, FP])
    

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0


In [65]:
df_at_R = pd.DataFrame(l_classifier_res, columns = ["R", "kappa", "accu", "TN", "FN", "TP", "FP"])
df_at_R.head(200)

,R,kappa,accu,TN,FN,TP,FP
0,0,0.0,0.000,0,0,0,200
1,0,0.1,0.015,3,0,0,197
2,0,0.2,0.040,8,0,0,192
3,0,0.3,0.085,17,0,0,183
4,0,0.4,0.195,39,0,0,161
5,0,0.5,0.370,74,0,0,126
6,0,0.6,0.560,112,0,0,88
7,0,0.7,0.690,138,0,0,62
8,0,0.8,0.835,167,0,0,33
9,0,0.9,0.910,182,0,0,18


In [61]:
if all(results_uni['5.0'] == 0) and all(np.array(Y_true) == 0):
    print("Zero")

Zero


In [50]:
confusion_matrix(Y_true, results_uni['5.0'])

array([[200]])

In [31]:
def multiply(a, b=2, c=1):
    return a * b *c

In [32]:
from multiprocessing import Pool
with Pool(5) as p:
    results = p.starmap(multiply, [(1,3), (2,3), (3,3)])
print(results)

[3, 6, 9]


In [25]:
a = [1,2,3]
b = [11, 12, 13]
c = [21, 22, 23]

df = pd.DataFrame(np.array([a,b,c]).T, columns= ["A", "B", "C"])
df.head()

,A,B,C
0,1,11,21
1,2,12,22
2,3,13,23


In [5]:
print(len(WL))
for clientw in WL:
    print(len(clientw))

10
3382346
3382346
3382346
3382346
3382346
3382346
3382346
3382346
3382346
3382346


In [21]:
for server_round in meta_metrics.keys():
    print(server_round)
    print(meta_metrics[server_round].keys())
    for client in range(10):
        print(f"Client {client}: Layers {len(meta_metrics[server_round][str(client)][0])}")

0
dict_keys(['3', '7', '4', '9', '5', '8', '6', '1', '0', '2'])
Client 0: Layers 8
Client 1: Layers 8
Client 2: Layers 8
Client 3: Layers 8
Client 4: Layers 8
Client 5: Layers 8
Client 6: Layers 8
Client 7: Layers 8
Client 8: Layers 8
Client 9: Layers 8
1
dict_keys(['4', '5', '9', '7', '3', '0', '2', '6', '1', '8'])
Client 0: Layers 8
Client 1: Layers 8
Client 2: Layers 8
Client 3: Layers 8
Client 4: Layers 8
Client 5: Layers 8
Client 6: Layers 8
Client 7: Layers 8
Client 8: Layers 8
Client 9: Layers 8
2
dict_keys(['7', '8', '1', '3', '0', '9', '5', '4', '2', '6'])
Client 0: Layers 8
Client 1: Layers 8
Client 2: Layers 8
Client 3: Layers 8
Client 4: Layers 8
Client 5: Layers 8
Client 6: Layers 8
Client 7: Layers 8
Client 8: Layers 8
Client 9: Layers 8
3
dict_keys(['6', '1', '5', '4', '2', '7', '0', '9', '8', '3'])
Client 0: Layers 8
Client 1: Layers 8
Client 2: Layers 8
Client 3: Layers 8
Client 4: Layers 8
Client 5: Layers 8
Client 6: Layers 8
Client 7: Layers 8
Client 8: Layers 8
Cli

In [87]:
clients = {1: 'b', 2: 'c', 0: 'a'}
keys = clients.keys()

In [88]:
for i in range(3):
    print(clients[i])

a
b
c


In [99]:
x = np.array([1,0,0])
cli_order = [int(x) for x in clients.keys()]
print(cli_order)

[1, 2, 0]


In [100]:
res = np.zeros(len(x))
for i, order in enumerate(cli_order):
    res[order] = x[i]
print(res)


[0. 1. 0.]
